Follolwing Ben Cloward Landscape Texture optimization [YouTube](https://www.youtube.com/watch?v=mEDoy-N1ODQ&list=PL78XDi0TS4lEApKzZgBwZaEAyr97EywSJ&index=2&ab_channel=BenCloward)

CR: `(R, G, B, Alpha) = (Albedo.R, Albedo.G, Albedo.B, Roughness)`

NOS: `(R, G, B, Alpha) = (Normal.R, Normal.G, Cavity * AO, Displacement)`

In [1]:
from PIL import Image, ImageChops
from glob import glob
import os

In [9]:
# Create CR image
def create_cr_noh(path_to_asset, output_format="tga", inv_grayscale=False):

    ms_assets={x.split('_')[-1].split('.')[0]:x for x in glob(os.path.join(ms_asset_dir, "*.jpg"))}
    ms_assets

    with Image.open(ms_assets['Albedo']) as albedo, Image.open(ms_assets['Roughness']) as roughness:
        bw_roughness = roughness.convert("L")
        if inv_grayscale:
            bw_roughness = ImageChops.invert(bw_roughness)
        
        albedo.putalpha(bw_roughness)
        out_name = ms_assets['Albedo'].replace('Albedo.jpg', f'CR_inv.{output_format}')
        albedo.save(out_name)

    # Create NOH image
    with Image.open(ms_assets['Normal']) as normal, \
    Image.open(ms_assets['Cavity']) as cavity, Image.open(ms_assets['AO']) as ao,\
    Image.open(ms_assets['Displacement']) as displacement:


        r = normal.getchannel(0)
        g = normal.getchannel(1)
        b = ImageChops.multiply(cavity, ao).convert('L')
        if inv_grayscale:
            b = ImageChops.invert(b) 
        a = displacement.convert("L")

        result = Image.merge('RGBA', (r, g, b, a))
        out_name = ms_assets['Albedo'].replace('Albedo.jpg', f'NOH_inv.{output_format}')
        result.save(out_name)
        



In [10]:
ms_asset_dir = os.path.abspath("Megascans/Downloaded/surface/moss_rock_ujqmejtr/")

create_cr_noh(ms_asset_dir)